In [102]:
import selenium
import requests
from bs4 import BeautifulSoup
from time import sleep

In [103]:
URL = "https://www.latamairlines.com/pe/es/ofertas-vuelos?origin=LIM&inbound=2023-02-28T17%3A00%3A00.000Z&outbound=2022-12-18T17%3A00%3A00.000Z&destination=MAD&adt=1&chd=0&inf=0&trip=RT&cabin=Economy&redemption=false&sort=RECOMMENDED"

In [104]:
from selenium import webdriver

In [105]:
agent = {"User-Agent":"Mozilla/5.0"}
r = requests.get(URL, agent)
s = BeautifulSoup(r.text, 'lxml')

In [107]:
# options = webdriver.FirefoxOptions()
# options.add_argument('--incognito')
driver = webdriver.Firefox(executable_path='driver/geckodriver.exe')

In [108]:
driver.get(URL)

In [109]:
sleep(1)
vuelos = driver.find_elements_by_xpath('//ol[@aria-label="Vuelos disponibles."]//li[@class="sc-bJTOcE dnJSKm"]')
vuelo = vuelos[2]

In [110]:
tiempo = vuelo.find_elements_by_xpath('.//div[contains(@class, "sc-ekHBYt")]/div[contains(@class, "flight-information")]/span[@class="sc-eTyWNx ilRvHO"]')
duracion = vuelo.find_element_by_xpath('.//div[contains(@class, "sc-ekHBYt")]/div[contains(@class, "flight-duration")]/span[@class="sc-bsVVwV jAgyHF"]').text

In [111]:
salida = tiempo[0].text
llegada = tiempo[1].text.replace("\n", " ")
print("Salida: {}\nLlegada: {}\nDuración: {}".format(salida, llegada, duracion))

Salida: 6:55
Llegada: 6:00 +1
Duración: 17 h 5 min


In [112]:
button_itinerary = vuelo.find_element_by_xpath('.//a[@class="sc-iwsKbI xkrrd"]')

In [113]:
button_itinerary.click()

In [114]:
segments = vuelo.find_elements_by_xpath("//div[contains(@class, 'path-info')]/parent::section")

In [115]:
segment = segments[0]

In [116]:
avion = segment.find_element_by_xpath('.//span[@class="airplane-code"]').text

In [117]:
salida = segment.find_element_by_xpath(".//div[contains(@class,'sc-ldcLGC')]")
llegada = segment.find_element_by_xpath(".//div[contains(@class,'sc-hTQSVH')]")

In [118]:
origen = salida.find_element_by_xpath('./div[@class="iataCode"]/span').text
tiempo = salida.find_element_by_xpath('./div[@class="iataCode"]/span[@class="time"]').text
aeropuerto = salida.find_element_by_xpath('.//span[@class="ariport-name"]').text

In [119]:
driver.find_element_by_xpath('//div[@class="MuiDialog-container MuiDialog-scrollPaper"]').click()

In [120]:
vuelo.click()

In [121]:
tabla_tarifas = driver.find_element_by_xpath('//ol[@aria-label="Tarifas disponibles."]')

In [122]:
monedas = tabla_tarifas.find_elements_by_xpath('.//span[contains(@class, "currency")]')
tarifas = tabla_tarifas.find_elements_by_xpath('.//span[@class="sc-ccvjgv ddOTqA"]')
tarifas = list(map(lambda x: x.text, tarifas))
monedas = list(map(lambda x: x.text, monedas))
dict(zip(tarifas, monedas))

{'1493.21': 'USD', '1694.99': 'USD', '4490.41': 'USD'}

In [207]:
def getPrecios(vuelo):
    '''
    Función que retorna los precios de un vuelo con el siguiente orden (tarifas, monedas, tiers).
    '''
    vuelo.click()
    tabla_tarifas = driver.find_element_by_xpath('//ol[@aria-label="Tarifas disponibles."]')
    monedas = tabla_tarifas.find_elements_by_xpath('.//span[contains(@class, "currency")]')
    tarifas = tabla_tarifas.find_elements_by_xpath('.//span[@class="sc-ccvjgv ddOTqA"]')
    tier = tabla_tarifas.find_elements_by_xpath('.//span[@class="sc-igaqVs kvfVRk"]')
    tarifas = list(map(lambda x: x.text, tarifas))
    monedas = list(map(lambda x: x.text, monedas))
    tier = list(map(lambda x: x.text, tier))
    button = vuelo.find_element_by_xpath('.//button[contains(@class,"MuiButton-disableElevation")]')
    button.click()
    return list(zip(tarifas, monedas, tier))

In [221]:
def getSecciones(segment):
    salida = segment.find_element_by_xpath(".//div[contains(@class,'sc-ldcLGC')]")
    llegada = segment.find_element_by_xpath(".//div[contains(@class,'sc-hTQSVH')]")
    avion = segment.find_element_by_xpath('.//span[@class="airplane-code"]').text
    duracion = segment.find_element_by_xpath('.//span[@class="ariport-name"]').text
    # avion = segment.find_element_by_xpath('.//span[@class="airplane-code"]').text
    # duration = segment.find_element_by_xpath('.//span[contains(@class, "pathInfo-duration")]/parent::*/span[@class="time"]').text
    return {'salida': getDatos(salida),
            'llegada': getDatos(llegada),
            'avion': avion,
            'duracion': duracion}

In [208]:
def getDatos(seccion):
    origen = seccion.find_element_by_xpath('./div[@class="iataCode"]/span').text
    tiempo = seccion.find_element_by_xpath('./div[@class="iataCode"]/span[@class="time"]').text
    aeropuerto = seccion.find_element_by_xpath('.//span[@class="ariport-name"]').text
    
    return {'origen': origen, 'tiempo': tiempo, 'aeropuerto': aeropuerto}

In [209]:
def getSegmentos(segmentos):
    a_list = [getSecciones(segmento) for segmento in segmentos]
    return a_list

In [233]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException

In [210]:
def getInfoVuelo(vuelo, delay=1):
    close_itinerary = None
    try:
        tiempo = vuelo.find_elements_by_xpath('.//div[contains(@class, "sc-ekHBYt")]/div[contains(@class, "flight-information")]/span[@class="sc-eTyWNx ilRvHO"]')
        duracion = vuelo.find_element_by_xpath('.//div[contains(@class, "sc-ekHBYt")]/div[contains(@class, "flight-duration")]/span[@class="sc-bsVVwV jAgyHF"]').text
        salida = tiempo[0].text
        llegada = tiempo[1].text.replace("\n", " ")
        
        precios = getPrecios(vuelo)
        button_itinerary = vuelo.find_element_by_xpath('.//a[@class="sc-iwsKbI xkrrd"]')
        button_itinerary.click()
        sleep(delay)
        segments = vuelo.find_elements_by_xpath("//div[contains(@class, 'path-info')]/parent::section")
        segments = getSegmentos(segments)
        
        return { 'Tiempo': {'Salida': salida, 'Llegada': llegada, 'Duración': duracion},
                'Precios': precios,
                'Segmentos': segments}
    except Exception as e:
        print(e)
    finally:
        close_itinerary = driver.find_element_by_xpath('//div[@class="MuiDialog-container MuiDialog-scrollPaper"]')
        close_itinerary.click()
    # finally:
    #     close_itinerary = driver.find_element_by_xpath('//button[@id="itinerary-modal-0-dialog-title-close"]')
    #     close_itinerary.click()

In [130]:
from tabulate import tabulate
from IPython.display import HTML

In [190]:
def pprint(info):
    style="pretty"
    string = ""
    string += ("Tiempo") + "\n"
    string += (tabulate(info['Tiempo'].items(), tablefmt=style)) + "\n"
    string += tabulate(info['Precios'], headers=['Precio', 'Moneda', 'Tier'], tablefmt=style) + "\n"
    for i in info['Segmentos']:
        string += (tabulate(i['salida'].items(), tablefmt=style)) + "\n"
        string += (tabulate(i['llegada'].items(), tablefmt=style)) + "\n"
        # string += (tabulate(zip([i['avion']], [i['duracion']]), headers=['avion', 'duracion'], tablefmt=style)) + "\n"
        return string

In [238]:
driver = webdriver.Firefox(executable_path='driver/geckodriver.exe')
driver.get(URL)
delay = 10
try:
    vuelo = WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.XPATH, '//li[@class="sc-bJTOcE dnJSKm"')))
    print("La pagina cargo")
except TimeoutException as te:
    print('La pagina se tardo demasiado')

La pagina se tardo demasiado


In [232]:
info = []
for idx, vuelo in enumerate(vuelos):
    info.append(getInfoVuelo(vuelo, delay=1.1))
    print(f"Vuelo {idx}")

Vuelo 0
Vuelo 1
Vuelo 2
Vuelo 3
Vuelo 4
Vuelo 5
Vuelo 6
Vuelo 7
Vuelo 8
Vuelo 9
Vuelo 10
Vuelo 11
Vuelo 12
Vuelo 13
Vuelo 14
Vuelo 15
Vuelo 16
Vuelo 17
Vuelo 18
Vuelo 19
Vuelo 20
Vuelo 21
Vuelo 22
Message: Element <li class="sc-bJTOcE dnJSKm"> is not clickable at point (408,470) because another element <div class="MuiDialog-container MuiDialog-scrollPaper"> obscures it

Vuelo 23
Vuelo 24
Vuelo 25
Vuelo 26
Vuelo 27
Vuelo 28
Message: Element <li class="sc-bJTOcE dnJSKm"> is not clickable at point (408,470) because another element <div class="MuiBackdrop-root"> obscures it

Vuelo 29


In [222]:
test = getInfoVuelo(vuelos[28])

In [226]:
test['Segmentos'][3]

{'salida': {'origen': 'GRU',
  'tiempo': '15:40',
  'aeropuerto': 'Guarulhos Intl.'},
 'llegada': {'origen': 'MAD', 'tiempo': '6:00', 'aeropuerto': 'Barajas Intl.'},
 'avion': 'Airbus A330',
 'duracion': 'Guarulhos Intl.'}

In [ ]:
driver.close()